In [1]:
import time 
import warnings 

import numpy as np
import pandas as pd

from sklearn import cluster
from sklearn.metrics import silhouette_score

from sklearn.preprocessing import StandardScaler

import plotly.plotly as py
import plotly.graph_objs as go

In [22]:
from sklearn.preprocessing import scale

In [23]:
from sklearn.preprocessing import MinMaxScaler

In [164]:
#from getUserStudyModel import getModel
#userModel=getModel()
userModel = pd.read_csv('users_prob_seq_model.csv', sep=',', encoding='utf-8')


In [165]:
userModel=userModel.drop_duplicates()

In [166]:
userModel.shape

(34, 36)

In [38]:
#userModel=userModel.loc[:,['problem_duration_graded','problem_duration_ungraded','video_duration','messagiesNum']].drop_duplicates()

In [37]:
#userModel1.to_csv('fully_study_model.csv',index=False)

In [167]:
def clustering(userModel):

    #X=userModel.loc[userModel['theme_id']=='e3c0abd8380d446c8e9a5fb6b15aeefc']
    X=userModel
    #X=X.loc[:,['problem_duration_graded','problem_duration_ungraded','video_duration']]    
    
    X = StandardScaler().fit_transform(X)
    #X = MinMaxScaler().fit_transform(X)
    #X = scale(X) 
    
    
    floatArr=[i for i in np.arange(0.1,1.1,0.1)]
    intArr=[i for i in range(2,10)]
    
    
    #eps float and minsamples int
    algorithm=cluster.DBSCAN()   
    
    #eigen_tol float and n_clusters int (optional)
    #algorithm=cluster.SpectralClustering() 
    
    #n_clusters int
    #algorithm=cluster.AgglomerativeClustering()
    
    #n_clusters int and batch_size int
    #algorithm=cluster.MiniBatchKMeans()
    
    #dim=3
    #dim=2
    
    coords=[]
    
    for para1 in floatArr:
    #for para1 in intArr:
        for para2 in intArr:
            silhouette,time,clusters_num=clusteringParams(X,para1,para2,algorithm)            
            coords.append((para1,para2,silhouette,time,clusters_num))
    """
    for para1 in intArr:
        silhouette,time,clusters_num=clusteringParams(X,para1,0,algorithm)
        coords.append((para1,silhouette,time))
        
    """
            
    return coords,3
    

In [168]:
def clusteringParams(X,para1,para2,algorithm):

    algorithm.set_params(eps=para1, min_samples=para2) #dbscan
    #algorithm.set_params(n_clusters=para2,eigen_tol=para1) #spectural
    #algorithm.set_params(n_clusters=para1) #agglomerative
    #algorithm.set_params(n_clusters=para1,batch_size=para2) #k-means
    
    t0 = time.time()
    
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore",
            message="the number of connected components of the " +
            "connectivity matrix is [0-9]{1,2}" +
            " > 1. Completing it to avoid stopping the tree early.",
            category=UserWarning)
        warnings.filterwarnings(
            "ignore",
            message="Graph is not fully connected, spectral embedding" +
            " may not work as expected.",
            category=UserWarning)
        algorithm.fit(X)
    t1 = time.time()
    
    labels_num=len(np.unique(algorithm.labels_))

    if len(np.unique(algorithm.labels_))>1:
        silhouette=silhouette_score(X,labels=algorithm.labels_)        
    else:
        silhouette=-1
       
    return silhouette, (t1-t0) , labels_num

In [169]:
coords,dim=clustering(userModel)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [148]:
tmp=pd.DataFrame(coords, columns=['para1','para2','silhouette','time','clusters_num'])
#tmp=pd.DataFrame(coords, columns=['para1','silhouette','time'])

In [149]:
maxCoef=max(list(tmp['silhouette']))
maxCoef

0.2931281961448429

In [150]:
t=tmp.loc[tmp['silhouette']==maxCoef]

In [10]:
minTime=min(list(t['time']))

In [11]:
t=t.loc[t['time']==minTime]

In [151]:
t

,para1,para2,silhouette,time,clusters_num
74,1.0,4,0.293128,0.000997,2
75,1.0,5,0.293128,0.000000,2
76,1.0,6,0.293128,0.000997,2
77,1.0,7,0.293128,0.000998,2
78,1.0,8,0.293128,0.000997,2


In [ ]:
#metricPlot(coords,dim)

In [ ]:
#скорость работы

In [ ]:
"""
userModel = pd.read_csv('problems_starts_time.csv', sep=',', encoding='utf-8')
#problems_starts_time
X=userModel.loc[:,['problem_duration_graded','problem_duration_ungraded','messagiesNum','video_duration']]    
X = StandardScaler().fit_transform(X)
"""

In [ ]:
"""
algorithm1=cluster.DBSCAN(eps=1.0, min_samples=3)
algorithm2=cluster.SpectralClustering(n_clusters=3,eigen_tol=0.9)
algorithm3=cluster.AgglomerativeClustering(n_clusters=2)
algorithm4=cluster.MiniBatchKMeans(n_clusters=2,batch_size=5)

algorithms=[algorithm1,algorithm2,algorithm3,algorithm4]
"""

In [ ]:
"""
algotime=[]
pointsX=[]
for algorithm in algorithms:
    tmpAlgoTime=[]
    for dataPercent in range(10,101,10):
        size=round(X.shape[0]*dataPercent/100)
        if len(pointsX)<10:
            pointsX.append(size)
        tmpX=X[0:size]
        silhouette, wtime = clusteringParams(tmpX,0,0,algorithm)
        tmpAlgoTime.append(wtime)
    algotime.append(tmpAlgoTime)
"""

In [ ]:
#df=pd.DataFrame(algotime).transpose()

In [ ]:
"""
names = ['DBSCAN','Spectral Clustering','Agglomerative clustering','K-means']

data=[]
for algoNum in range(df.shape[1]):
    Y=df[algoNum]
    data.append(
        go.Scatter(
            x=pointsX,
            y=Y,
            mode='markers+lines',
            marker=dict(size=5),
            name=names[algoNum]
        )   
    )


layout = go.Layout(        
        xaxis=dict(title='Количество данных для обучения',ticklen= 5, showticklabels=True),
        yaxis=dict(title='Время, потраченное на обучение (в секундах)',ticklen= 5,zeroline= True,showticklabels=True),
    legend=dict(x=0.01, y=1)
    
    )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, world_readable=True, filename='specStudyModel')
"""

In [ ]:
#основные кластеры

In [ ]:
Поменять параметры в моделях для конкретной задаче

In [ ]:
algorithm1=cluster.DBSCAN(eps=1.0, min_samples=3)
algorithm2=cluster.SpectralClustering(n_clusters=3,eigen_tol=0.9)
algorithm3=cluster.AgglomerativeClustering(n_clusters=2)
algorithm4=cluster.MiniBatchKMeans(n_clusters=2,batch_size=5)

algorithms=[algorithm1,algorithm2,algorithm3,algorithm4]

In [16]:

#userModel = pd.read_csv('problems_starts_time.csv', sep=',', encoding='utf-8')
#problems_starts_time
models=userModel  
scal = StandardScaler()
X_t = scal.fit_transform(models)

#centers=scal.inverse_transform(algorithm3.cluster_centers_)

In [152]:
algorithm1=cluster.DBSCAN(eps=1.0, min_samples=8)
algorithm1.fit(X_t)

DBSCAN(algorithm='auto', eps=1.0, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=8, n_jobs=1, p=None)

In [38]:
algorithm2=cluster.SpectralClustering(n_clusters=2,eigen_tol=0.5)
algorithm2.fit(X_t)

SpectralClustering(affinity='rbf', assign_labels='kmeans', coef0=1, degree=3,
          eigen_solver=None, eigen_tol=0.5, gamma=1.0, kernel_params=None,
          n_clusters=2, n_init=10, n_jobs=1, n_neighbors=10,
          random_state=None)

In [57]:
algorithm3=cluster.AgglomerativeClustering(n_clusters=2)
algorithm3.fit(X_t)

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='ward', memory=None, n_clusters=2,
            pooling_func=<function mean at 0x000001F2712EC9D8>)

In [99]:
algorithm4=cluster.MiniBatchKMeans(n_clusters=2,batch_size=6)
algorithm4.fit(X_t)

MiniBatchKMeans(batch_size=6, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10, n_clusters=2,
        n_init=3, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=0)

In [ ]:
"""
X=userModel.loc[:,['problem_duration_graded','problem_duration_ungraded','video_duration']]
X = scale(X) 
algorithm2.fit(X)
"""

In [ ]:
"""
from sklearn.preprocessing import MinMaxScaler 
scale_features = MinMaxScaler() 
X=userModel.loc[:,['problem_duration_graded','problem_duration_ungraded','messagiesNum','video_duration']]  
features_train = scale_features.fit_transform(X) 
algorithm1.fit(features_train)
"""

In [ ]:
centers=scal.inverse_transform(algorithm4.cluster_centers_)
centers

In [153]:
tmp=pd.DataFrame(algorithm1.labels_)

In [154]:
labeledModel=userModel.join(tmp)

In [156]:
labeledModel.columns

Index(['TS_V', 'TS_T', 'TS_P',  'V_V',  'V_T',  'V_P', 'V_TE',  'T_V',  'T_T',
        'T_P', 'T_TE',  'P_V',  'P_T',  'P_P', 'P_TE',      0],
      dtype='object')

In [157]:
#forcalc=labeledModel.loc[labeledModel[0]==0]
minimum=labeledModel.groupby(0)[labeledModel.columns].min()
maximum=labeledModel.groupby(0)[labeledModel.columns].max()


In [159]:
number=labeledModel.groupby(0)[labeledModel.columns].count()

In [160]:
number

,TS_V,TS_T,TS_P,V_V,V_T,V_P,V_TE,T_V,T_T,T_P,T_TE,P_V,P_T,P_P,P_TE,0
0,,,,,,,,,,,,,,,,
-1,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
0,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [161]:
minimum

,TS_V,TS_T,TS_P,V_V,V_T,V_P,V_TE,T_V,T_T,T_P,T_TE,P_V,P_T,P_P,P_TE,0
0,,,,,,,,,,,,,,,,
-1,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
0,0.7,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1.0,0.0,0.0,0.384615,0.0,0.0,0.615385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [162]:
#labeledModel.loc[labeledModel[0]==0].loc[labeledModel['problem_duration_graded']==0].loc[labeledModel['problem_duration_ungraded']==0].loc[labeledModel['video_duration']==0]

In [163]:
maximum

,TS_V,TS_T,TS_P,V_V,V_T,V_P,V_TE,T_V,T_T,T_P,T_TE,P_V,P_T,P_P,P_TE,0
0,,,,,,,,,,,,,,,,
-1,1.0,0.000000,0.1,0.818182,0.0,0.235294,0.588235,0.0,0.000000,0.0,0.0,6.0,0.0,0.0,1.0,-1
0,1.0,0.111111,0.3,0.823529,1.0,0.230769,1.000000,1.0,0.703704,0.0,1.0,19.0,0.0,8.0,3.0,0
1,1.0,0.000000,0.0,0.384615,0.0,0.000000,0.615385,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
def metricPlot(coords,dim):
    marker=dict(
        size=5,
        line=dict(
            color='rgb(0, 0, 0)',
            width=0.5
        )
    )
    
    if dim==3:
        tmp=pd.DataFrame(coords, columns=['para1','para2','coefficient','time','cluster_num'])
        X=tmp['para1']
        Y=tmp['para2']
        Z=tmp['coefficient']
        #text="time: "+ str(tmp['time'])+", clusters: "+ str(tmp['cluster_num'])
        
        trace = go.Scatter3d(
            x=X,
            y=Y,
            z=Z,
            text=tmp['cluster_num'],
            mode='markers',
            marker=marker
        )
    else:
        tmp=pd.DataFrame(coords, columns=['para1','coefficient','time'])
        X=tmp['para1']
        Y=tmp['coefficient']
        text=tmp['time']
        
        trace = go.Scatter(
            x=X,
            y=Y,
            text=text,
            mode='markers',
            marker=marker
        )         

    data = [trace]
    
    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        ),
        scene = dict(
        xaxis=dict(title='eps'),
        yaxis=dict(title='min_samples'),
        zaxis=dict(title='silhouette')
        )
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, world_readable=True, filename='dbscan')

In [ ]:
marker=dict(
    size=5,
        line=dict(
            color='rgb(0, 0, 0)',
            width=0.5
        )
)
        
trace = go.Scatter3d(
    x=tmp['para1'],
    y=tmp['para2'],
    z=tmp['silhouette'],
    text=tmp['clusters_num'],
    mode='markers',
    marker=marker
)
data = [trace]
    
layout = go.Layout(
    margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
    ),
    scene = dict(
        xaxis=dict(title='eps'),
        yaxis=dict(title='min_samples'),
        zaxis=dict(title='silhouette')
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, world_readable=True, filename='dbscan')